## User Profile

In [1]:
import pandas as pd
import json
import csv
import statistics

In [2]:
DATASET_BASE = "../fakenewsnet_dataset"
NEWS_PATH = f"{DATASET_BASE}/politifact/fake"

In [3]:
train_id_df = pd.read_csv("../exported/train_ids.csv", header = 0)
cv_id_df = pd.read_csv("../exported/cv_ids.csv", header = 0)
test_id_df = pd.read_csv("../exported/test_ids.csv", header = 0)

In [4]:
uids = []

# text
descriptions = []
locations = []

# numeric
followers_counts = []
friends_counts = []
listed_counts = []
favorites_counts = []
statuses_counts = []

# boolean
protecteds = []
geo_enableds = []
verifieds = []

for idx, row in train_id_df.iterrows():
    
    if idx % 100 == 0:
        print(".", end="")
    
    with open(f"{NEWS_PATH}/{row['news_id']}/tweets/{row['source_tweet_id']}.json", "r") as f:
        t = json.loads(f.read())
        u = t['user']
        
        uids.append(u['id'])
        descriptions.append(u['description'])
        locations.append(u['location'])
        followers_counts.append(u['followers_count'])
        friends_counts.append(u['friends_count'])
        listed_counts.append(u['listed_count'])
        favorites_counts.append(u['favourites_count'])
        statuses_counts.append(u['statuses_count'])
        protecteds.append(u['protected'])
        geo_enableds.append(u['geo_enabled'])
        verifieds.append(u['verified'])

.........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [5]:
for idx, des in enumerate(descriptions):
    descriptions[idx] = des.replace("\r\n", " ").replace("\n", " ")
    
for idx, loc in enumerate(locations):
    locations[idx] = loc.replace("\r\n", " ").replace("\n", " ")

In [6]:
train_id_df['uid'] = uids
train_id_df['description'] = descriptions
train_id_df['location'] = locations
train_id_df['protected'] = protecteds
train_id_df['geo_enabled'] = geo_enableds
train_id_df['verified'] = verifieds
train_id_df['followers_count'] = followers_counts
train_id_df['friends_count'] = friends_counts
train_id_df['listed_count'] = listed_counts
train_id_df['favorites_count'] = favorites_counts
train_id_df['statuses_count'] = statuses_counts

In [7]:
train_df = train_id_df[[
    'news_id', 'source_tweet_id', 'uid',
    'description', 'location',
    'protected', 'geo_enabled', 'verified',
    'followers_count', 'friends_count', 'listed_count', 'favorites_count', 'statuses_count',
    'cascade_size']]

In [8]:
train_df.to_csv("../exported/user_profile_train.csv", header = True, index = False, quoting = csv.QUOTE_NONNUMERIC, quotechar='"')

## Check

In [14]:
with open("../exported/user_profile_test.csv", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f.readlines()):
        if line[:5] != '"poli':
            print(idx)

0


## User Timeline

In [4]:
train_id_df = pd.read_csv("../exported/train_ids.csv", header = 0)
cv_id_df = pd.read_csv("../exported/cv_ids.csv", header = 0)
test_id_df = pd.read_csv("../exported/test_ids.csv", header = 0)

In [5]:
K = 10

In [18]:
historical_texts = []
avg_rt_counts = []
avg_favorite_counts = []
sensitive_ratios = []

for idx, row in train_id_df.iterrows():
    
    recent_posts = []
    
    if idx % 100 == 0:
        print(".", end="")
    
    with open(f"{NEWS_PATH}/{row['news_id']}/tweets/{row['source_tweet_id']}.json", "r") as f:
        t = json.loads(f.read())
        tid = t['id']
        u = t['user']
        
        with open(f"{DATASET_BASE}/user_timeline_tweets/{u['id']}.json", "r") as ff:
            try:
                posts = json.loads(ff.read())
            except:
                historical_texts.append("")
                avg_rt_counts.append(-1)
                avg_favorite_counts.append(-1)
                sensitive_ratios.append(-1)
                continue
            
            total_n = len(posts)
            target_at = -1
            
            for i, p in enumerate(posts):
                if tid == p['id']:
                    target_at = i
                    break
            
            if target_at != -1:
                if (total_n - 1 - target_at) >= K:
                    recent_posts = posts[target_at+1:target_at+1+K]
            elif total_n >= K:
                recent_posts = posts[total_n-K:]
            else:
                recent_posts = posts

            if len(recent_posts):
                concat_text = ""
                rt_counts = []
                favorite_counts = []
                sensitive_count = 0
                for p in recent_posts:
                    concat_text += p['text']
                    rt_counts.append(p['retweet_count'])
                    favorite_counts.append(p['favorite_count'])
                    if "possibly_sensitive" in p and p['possibly_sensitive']:
                        sensitive_count += 1

                historical_texts.append(concat_text)
                avg_rt_counts.append(statistics.mean(rt_counts))
                avg_favorite_counts.append(statistics.mean(favorite_counts))
                sensitive_ratios.append(sensitive_count / len(recent_posts))
            else:
                historical_texts.append("")
                avg_rt_counts.append(-1)
                avg_favorite_counts.append(-1)
                sensitive_ratios.append(-1)

.........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [19]:
for idx, t in enumerate(historical_texts):
    historical_texts[idx] = t.replace("\r\n", " ").replace("\n", " ")

In [20]:
train_id_df['text'] = historical_texts
train_id_df['avg_rt_count'] = avg_rt_counts
train_id_df['avg_favorite_count'] = avg_favorite_counts
train_id_df['sensitive_ratio'] = sensitive_ratios

train_df = train_id_df[['news_id', 'source_tweet_id',
                      'text', 'avg_rt_count', 'avg_favorite_count', 'sensitive_ratio',
                      'cascade_size']]

train_df.to_csv("../exported/user_timeline_train.csv", header = True, index = False, quoting = csv.QUOTE_NONNUMERIC, quotechar='"')

Check:

In [32]:
with open("../exported/user_timeline_test.csv", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f.readlines()):
        if line[:5] != '"poli':
            print(idx)

0
